In [121]:
from laplace import Laplace

In [1]:
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(42)

# Load data from CSV file using pandas
df = pd.read_csv('data1.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy())
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

tensor([[ 2.2603,  5.3383],
        [ 4.2518,  5.0986],
        [ 2.1183,  3.8116],
        [ 3.0115,  2.3589],
        [ 2.9008,  4.8617],
        [ 5.1086,  5.1326],
        [ 2.5563,  1.6711],
        [ 4.4466,  3.8073],
        [ 2.6584,  1.7919],
        [ 2.6213,  4.9914],
        [ 2.4809,  4.1302],
        [ 1.1887,  5.2070],
        [ 2.4869,  4.3043],
        [ 3.4805,  5.0592],
        [ 3.8918,  3.2124],
        [ 3.5684,  3.9414],
        [ 3.6159,  3.6186],
        [ 1.7725,  1.4366],
        [ 4.0901,  3.2862],
        [ 3.2045,  1.9775],
        [ 4.7188,  2.3002],
        [ 1.3007,  2.3323],
        [ 1.2066,  3.3627],
        [ 3.9632,  3.4090],
        [ 3.3917,  1.2179],
        [-2.6758, -2.8583],
        [-5.2805, -3.3999],
        [-3.6253, -3.6108],
        [-6.0946, -5.4988],
        [-3.9991, -5.1169],
        [-4.9641, -3.6346],
        [-4.4539, -6.0210],
        [-4.6465, -5.4079],
        [-5.5234, -2.7017],
        [-3.2513, -5.4246],
        [-2.7528, -2

In [123]:
X = torch.tensor(x.T).float().T

# Convert y to a tensor of indices and one-hot encode it
y_unique = torch.unique(torch.tensor(y))
y_indices = torch.searchsorted(y_unique, torch.tensor(y))
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18860\1740726160.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(x.T).float().T
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18860\1740726160.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_unique = torch.unique(torch.tensor(y))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18860\1740726160.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_indices = torch.searchsorted(y_unique, torch.tensor(y))


In [124]:
# Define neural network
n_hidden = 3
D = X.shape[1]
out_dim = y_train.shape[1]
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

In [125]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [126]:
# Train the model
epochs = 2500
show_every = epochs // 10

for epoch in range(1, epochs+1):
    loss_sum = 0
    y_pred = model(X)
    loss = criterion(y_pred, y_train)
    loss_sum += loss.item()
    loss.backward()
    optimizer.step()
    
    if epoch % show_every == 0:
        print("Epoch ", epoch)
        print("Loss: ", loss_sum)


Epoch  250
Loss:  1.1131099462509155
Epoch  500
Loss:  0.8203388452529907
Epoch  750
Loss:  0.5208479166030884
Epoch  1000
Loss:  0.329197496175766
Epoch  1250
Loss:  0.21610870957374573
Epoch  1500
Loss:  0.14689311385154724
Epoch  1750
Loss:  0.10169566422700882
Epoch  2000
Loss:  0.07326287031173706
Epoch  2250
Loss:  0.05266139656305313
Epoch  2500
Loss:  0.03975914791226387


In [127]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full')

In [128]:
la.fit(DataLoader(TensorDataset(X, y_train)))

In [129]:
la.optimize_prior_precision()

In [130]:
probit_predictions = la(X, link_approx='probit')

In [131]:
predictions_probit_df = pd.DataFrame(probit_predictions.numpy(), columns=['class1', 'class2', 'class3', 'class4'])

In [132]:
predictions_probit_df.to_csv('predictions1-Python.csv', index=False)

In [133]:
print(probit_predictions)

tensor([[0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7583, 0.0355, 0.1054, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7586, 0.0354, 0.1052, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7557, 0.0356, 0.1067, 0.1020],
        [0.7589, 0.0353, 0.1049, 0.1009],
        [0.7587, 0.0354, 0.1050, 0.1009],
        [0.7588, 0.0354, 0.1050, 0.1009],
        [0.7582, 0.0353, 0.1052, 0.1013],
        [0.7588, 0.0353, 0.1049, 0.1010],
        [0.7589, 0.0353, 0.1049, 0

In [134]:
torch.set_printoptions(sci_mode=False)
print(torch.softmax(model(X), dim=1))

tensor([[    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9938,     0.0001,     0.0032,     0.0029],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0032,     0.0029],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0.9939,     0.0001,     0.0031,     0.0028],
        [    0